In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import os
import time
import re

action_name = ""

def scrape(auctionId):
    path= f"https://www.cityauctiongroup.com/auction/{auctionId}"
    global action_name
    options = ChromeOptions()
    options.headless = False
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()



    # ---- LOGIN ----
    try:
        login = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, './/a[text()="Login"]'))
        )
        login.click()

        user = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, 'username'))
        )
        user.send_keys("fourbrotherstrading@icloud.com")

        pswd = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, 'password'))
        )
        pswd.send_keys("Sultanmirza1501#")

        signin = driver.find_element(By.XPATH, './/button[text()="Sign in"]')
        signin.click()
    except:
        pass

    time.sleep(2)

    try:
        action_name = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="col-md-8"]/h1'))
        ).text.strip()
        print(f"Auction Name: {action_name}")
    except:
        pass
    page_links = driver.find_elements(By.XPATH, '//div[@class="motorvehicle-pagination-buttons"]/a')
    if page_links:
        page_numbers = []
        for link in page_links:
            text = link.text.strip()
            if text.isdigit():
                page_numbers.append(int(text))

        last_page = max(page_numbers)
        print(f"📌 Total pages found: {last_page}")
    else:
        last_page = 1
        print("📌 Only one page found")

    # ---- Create folder ----
    html_folder = "html"
    os.makedirs(html_folder, exist_ok=True)

    car_count = 0

    # ---- LOOP THROUGH PAGES ----
    for page in range(1, last_page + 1):

        page_url = f"{path}?page={page}"
        print(f"\n📄 Opening Page {page} -> {page_url}")

        driver.get(page_url)
        time.sleep(2)

        # Fetch cars on this page
        cars = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[@class="row mb--"]'))
        )

        print(f"➡ Cars on page: {len(cars)}")

        for i in range(len(cars)):
            cars = driver.find_elements(By.XPATH, '//div[@class="row mb--"]')

            try:
                driver.execute_script("arguments[0].scrollIntoView();", cars[i])
                time.sleep(1)
                cars[i].click()
                time.sleep(2)

                # REG
                try:
                    reg = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located(
                            (By.XPATH, './/table//tr[th[text()="Registration"]]/td')
                        )
                    ).text.strip()
                except:
                    reg = f"car_{car_count+1}"

                html = driver.page_source
                with open(os.path.join(html_folder, f"{reg}.html"), "w", encoding="utf-8") as f:
                    f.write(html)

                car_count += 1
                print(f"✔ Saved {reg}  ({car_count})")

                driver.back()
                time.sleep(2)

            except Exception as e:
                print("❌ Could not save car:", e)
                driver.back()
                time.sleep(1)
                continue

    driver.quit()
    print(f"\n🎉 Total cars saved: {car_count}")

scrape(1996)

Auction Name: Non Runner Sale Event Auction
📌 Only one page found

📄 Opening Page 1 -> https://www.cityauctiongroup.com/auction/1996?page=1
➡ Cars on page: 7
✔ Saved BT20ONV  (1)
✔ Saved HW70XWE  (2)
✔ Saved HW70XVN  (3)
✔ Saved MC21ULK  (4)
✔ Saved CN65PXM  (5)
✔ Saved BJ67YMP  (6)
✔ Saved YK20XWM  (7)

🎉 Total cars saved: 7


In [2]:
import os,re,json,csv
from bs4 import BeautifulSoup
# action_name ="Test" 
html_folder = "html"
output_file = "data.csv"

headers = [
    "Auction Name",
    "Title",
    "Lot",
    "Center",
    "Start Date",
    "Start Time",
    "D.O.R",
    "Manufacturer",
    "Model",
    "Variant",
    "Mileage",
    "Mileage Warranted",
    "CC",
    "Colour",
    "Former Keepers",
    "Transmission",
    "Fuel Type",
    "Reg",
    "Body type",
    "V5",
    "VAT Status",
    "Parcel Shelf",
    "Keys",
    "Vendor",
    "MOT Due",
    "CAP Clean",
    "CAP Average",
    "CAP Below",
    "Date/Time",
    "Inspection Report",
    "Grade",
    "Features",
    "Non Runner",
    "Brakes",
    "Euro Status",
    "Tyres Condition",
    "Additional Information",
    "General Condition",
    "Service History",
    "No of Services",
    "Last Service",
    "Last Service Mileage",
    "Images",
    "Damaged_images",
    "Damage_details",
    "Notes",

]

with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    previous_data = {
        "Center": "",
        "Start Date": "",
        "Start Time": ""
    }
    for file_name in os.listdir(html_folder):
        if not file_name.endswith(".html"):
            continue

        file_path = os.path.join(html_folder, file_name)

        with open(file_path, "r", encoding="utf-8") as f_html:
            soup = BeautifulSoup(f_html.read(), "html.parser")

        def get_value(label):
            try:
                row = soup.find("th", string=lambda x: x and x.strip() == label).find_parent("tr")
                return row.find("td").get_text(strip=True)
            except:
                return ""

            
        raw_mileage = get_value("Mileage") 

        cleaned_mileage = str(raw_mileage).replace(",", "")

        # Extract number
        mileage_num = ""
        match = re.search(r'\d+', cleaned_mileage)
        if match:
            mileage_num = int(match.group(0))

        # Warranted check
        mileage_warranted = "Yes" if "Warranted" in raw_mileage else "No"

            
        raw_cc = get_value("CC")  

        try:
            cc_num = round(float(raw_cc.replace(",", "")) / 1000, 1)  
        except:
            cc_num = ""

        data = {
            "Auction Name":action_name if action_name else "",
            "D.O.R": get_value("Registered"),
            "Manufacturer": get_value("Manufacturer"),
            "Model": get_value("Model"),
            "Variant": get_value("Variant"),
            "Mileage": mileage_num,
            "Mileage Warranted": mileage_warranted,
            "CC": cc_num,
            "Colour": get_value("Colour"),
            "Former Keepers": get_value("Previous Owners"),
            "MOT Due": get_value("MOT Due"),
            "Transmission": get_value("Transmission"),
            "Fuel Type": get_value("Fuel"),
            "Reg": get_value("Registration"),
            "Body type": get_value("Body type"),
            "V5": get_value("V5"),
            "VAT Status": get_value("VAT"),
            "Parcel Shelf": get_value("Parcel Shelf"),
            "Keys": get_value("Keys"),
            "Vendor": get_value("Vendor"),
            "CAP Clean": get_value("CAP Clean"),
            "CAP Average": get_value("CAP Average"),
            "CAP Below": get_value("CAP Below"),
            "Date/Time": get_value("Date/Time"),
        }
        title_div = soup.find("div", {"class": "vehicle-title"})

        if title_div:
            try:
                h1_text = title_div.find("h1").get_text(strip=True) 
            except:
                h1_text = ""

            lot = ""
            main_title = h1_text

            if ":" in h1_text:
                parts = h1_text.split(":", 1)
                lot_part = parts[0].strip()          
                main_title = parts[1].strip() 

                match = re.search(r'\d+', lot_part)
                if match:
                    lot = match.group(0)         
                else:
                    lot = ""

        else:
            lot = ""
            main_title = ""

        data["Lot"] = lot
        data["Title"] = main_title



   


        actions_div = soup.find("div", {"class": "vehicle-actions"})


        center = ""
        date = ""
        time_ = ""

        if actions_div:
            spans = actions_div.find_all("span", style=True) 
            
        
            for span in spans:
                text_content = span.get_text(strip=True)
                
          
                if span.find("svg", {"class": "feather-map-pin"}):
                    center = text_content
                    
          
                elif span.find("svg", {"class": "feather-calendar"}):
                    date = text_content
                    
  
                elif span.find("svg", {"class": "feather-clock"}):
                    time_ = text_content
                
            if center and date and time_:

                previous_data["Center"] = center
                previous_data["Start Date"] = date
                previous_data["Start Time"] = time_
            
        if not center:

            center = previous_data["Center"] 

        if not date:
            date = previous_data["Start Date"]
        if not time_:
            time_ = previous_data["Start Time"]
        data["Center"] = center
        data["Start Date"] = date
        data["Start Time"] = time_



        features_h2 = soup.find("h2", string=lambda x: x and "Features" in x)

        features_list = []

        if features_h2:
            features_div = features_h2.find_next_sibling("div", class_="row mb+")
            
            if features_div:
                tds = features_div.find_all("td")
                for td in tds:
                    text = td.get_text(strip=True)
                    if text:
                        features_list.append(text)


        features_csv = ", ".join(features_list)
        data["Features"] = features_csv
        
        additional_information = {}
        mech_h2 = soup.find("h2", string=lambda x: x and "Mechanical Inspection" in x)

        if mech_h2:
            row_div = mech_h2.find_next_sibling("div", class_="row mb+")
            if row_div:
                cols = row_div.find_all("div", class_="col-md-4")
                for col in cols:
                    section_name_tag = col.find("h4")
                    if not section_name_tag:
                        continue
                    section_name = section_name_tag.get_text(strip=True)

                    section_data = {}
                    table = col.find("table")
                    if table:
                        for tr in table.find_all("tr"):
                            th = tr.find("th")
                            td = tr.find("td")
                            if th and td:
                                key = th.get_text(strip=True)
                                val = td.get_text(strip=True)
                                section_data[key] = val
                    additional_information[section_name] = section_data
                    
        additional_information = additional_information or {}

        economy_data = {}
        economy_h2 = soup.find("h2", string="Economy")

        if economy_h2:
            parent_div = economy_h2.find_next_sibling("div", class_="row")
            if parent_div:
                tables = parent_div.find_all("table")
                for table in tables:
                    for tr in table.find_all("tr"):
                        th = tr.find("th")
                        td = tr.find("td")
                        if th and td:
                            key = th.get_text(strip=True)
                            value = td.get_text(strip=True)
                            economy_data[key] = value
        additional_information["Economy"] = economy_data


        data['General Condition'] = additional_information
        engine_section = additional_information.get("Engine", {})   
        engine_runs_value = engine_section.get("Engine Runs", "")  
        Nonrunner = "No" if engine_runs_value == "OK" else "Yes"
        data['Non Runner'] = Nonrunner
        
        
        # Brakes
        Brakes = engine_section.get("Brakes", "")
        data["Brakes"] = Brakes  # agar alag column bhi chahiye

        # Tyres
        tyres_list = []
        tyres_h4 = soup.find("h4", string=lambda x: x and "Tyres" in x)

        if tyres_h4:
            table = tyres_h4.find_next_sibling("table")
            if table:
                rows = table.find_all("tr")[1:]  # skip header
                for row in rows:
                    cols = row.find_all("td")
                    if len(cols) >= 3:
                        tyre_name = cols[0].get_text(strip=True)
                        make = cols[1].get_text(strip=True)
                        cond = cols[2].get_text(strip=True)
                        tyres_list.append(f"{tyre_name}: - {make if make else 'OTHER / UNBRANDED'}, {cond}")

        # Brakes ko tyres_list me add karna
        if Brakes:
            tyres_list.append(f"Brakes: - {Brakes}")

        # CSV me join karna
        tyres_csv = " | ".join(tyres_list)
        data["Tyres Condition"] = tyres_csv

        
        try:
            pdf_link = soup.find("a", string="View PDF")["href"]
        except:
            pdf_link = ""

        
        try:
            grade_img = soup.find("img", {"class": "img-responsive mb--"})["src"]
        except:
            grade_img = ""

        grade = ""
        if grade_img:
            match = re.search(r'/(\d+)\.(jpg|png|jpeg)', grade_img)
            if match:
                grade = match.group(1)


        
        
        interinfo = {}

        # Economy
        economy = {}
        h2 = soup.find("h2", string=lambda x: x and "Economy" in x)
        if h2:
            next_div = h2.find_next_sibling("div")
            if next_div:
                tables = next_div.find_all("table")
                for table in tables:
                    for tr in table.find_all("tr"):
                        key = tr.find("th").text.strip()
                        value = tr.find("td").text.strip()
                        economy[key] = value
        interinfo["economy"] = economy

        # Interior
        interior = {}
        h4 = soup.find("h4", string=lambda x: x and "Interior" in x)
        if h4:
            table = h4.find_next_sibling("table")
            if table:
                for tr in table.find_all("tr"):
                    th = tr.find("th")
                    td = tr.find("td")
                    if th and td:  # check if both exist
                        key = th.get_text(strip=True)
                        value = td.get_text(strip=True)
                        interior[key] = value
        interinfo["Interior"] = interior

        data["Additional Information"] = interinfo
        euro_stsus = economy["Standard Euro Emissions"]
        euro_number= ""
        matchh = re.search(r'\d+',euro_stsus)
        if matchh:
            euro_number = matchh.group(0)
            
        data["Euro Status"] = euro_number
        
       
        
        # Notes
        notes_panel = soup.find("div", class_="panel panel-danger")
        notes_text = ""

        if notes_panel:
            body = notes_panel.find("div", class_="panel-body")
            if body:
                notes_text = body.get_text(separator=" ", strip=True)  
        data["Notes"] = notes_text
        
        
        # Service History
        service_history_div = soup.find("h2", string="Service History")
        service_data = {
            "Service History": "",
            "Last Service": "",
            "Last Service Mileage": "",
            "No of Services": ""
        }

        if service_history_div:
            row_div = service_history_div.find_next("div", class_="row mb+")
            if row_div:
                cols = row_div.find_all("div", class_="col-md-4")
                for col in cols:
                    label = col.find("label").get_text(strip=True) if col.find("label") else ""
                    value = col.find("p").get_text(strip=True) if col.find("p") else ""
                    
                    if "Last Service Date" in label:
                        service_data["Last Service"] = value
                    elif "Number of Stamps" in label:
                        service_data["No of Services"] = value
                    elif "Last Service Miles" in label:
                        service_data["Last Service Mileage"] = value
      
                service_data["Service History"] = ", ".join([
                    service_data["Last Service"], 
                    service_data["Last Service Mileage"], 
                    service_data["No of Services"]
                ])


        data["Service History"] = ""
        data["Last Service"] = service_data.get("Last Service", "")
        data["No of Services"] = service_data.get("No of Services", "")
        data["Last Service Mileage"] = service_data.get("Last Service Mileage", "")

        
        # images
        image_urls = []
        for thumb_div in soup.find_all("div", class_="thumb"):
            href = thumb_div.get("href")
            if href:
                href = href.replace("200-150.jpg", "1604-1201.jpg")
                image_urls.append(href)

        image_urls_str = ", ".join(image_urls)
        data["Images"]=image_urls_str
        
        # Damage Details
        damage_details = []

        for desc_div in soup.select("div.gallerywidget-description"):
            content_div = desc_div.select_one("div.description-item-content")
            if content_div:
                texts = [t.strip() for t in content_div.stripped_strings]
                damage_details.append("|".join(texts))
        DAMAGE_DETAILS = ", ".join(damage_details)
        data["Damage_details"]=DAMAGE_DETAILS
        
        # Damaged Images
        damaged_images = []


        condition_div = soup.find("h2", text="Condition Check")
        if condition_div:

            thumbs_div = condition_div.find_next("div", class_="thumbs")
            if thumbs_div:
                for thumb_div in thumbs_div.select("div.thumb"):
                    href = thumb_div.get("href")
                    if href:
                 
                        href = href.replace("200-150.jpg", "1604-1201.jpg")
                        damaged_images.append(href)


        DAMAGED_IMAGES = ", ".join(damaged_images)
        data['Damaged_images'] = DAMAGED_IMAGES
                
                
        
        data["Inspection Report"] =f"https://www.cityauctiongroup.com/{pdf_link}"
        data["Grade"] = grade
        writer.writerow([data[h] for h in headers])

    




C:\Users\HP\AppData\Local\Temp\ipykernel_18272\964366724.py:444: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  condition_div = soup.find("h2", text="Condition Check")


In [3]:
import pandas as pd

csv_path = "data.csv"
df = pd.read_csv(csv_path)
df["Lot"] = pd.to_numeric(df["Lot"], errors="coerce")
df_sorted = df.sort_values(by="Lot", ascending=True)
df_sorted.to_csv("cag_data.csv", index=False)

print("CSV sorted and overwritten successfully!")


CSV sorted and overwritten successfully!


In [4]:
from PIL import Image, ImageDraw, ImageFont
def add_watermark_to_image(image_path, text="Sourced from City Auction Group"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # Load font
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_width = text_width + 2 * margin
        box_height = text_height + 2 * margin
        draw.rectangle([x - margin, y - margin, x - margin + box_width, y - margin + box_height],fill=(0, 0, 0, 160))

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 200))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"Watermark added to {image_path}")

    except Exception as e:
        print(f"Failed to watermark {image_path}: {e}")


In [5]:
import requests,threading
from urllib.parse import urlparse, urljoin
df = pd.read_csv("cag_data.csv")
reg_img = df[["Reg", "Images"]]
cond_img = df[["Reg", "Damaged_images"]]
report = df[["Reg",'Inspection Report']]

def download_images(data, main_folder="Images"):
    

    os.makedirs(main_folder, exist_ok=True)
    

    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        
        if pd.isna(row["Images"]) or not str(row["Images"]).strip():
            print(f"Skipping {reg_no} due to missing images.")
           
        else:
        
            image_urls = row["Images"].split(", ") 
            
    
            reg_folder = os.path.join(main_folder, reg_no) 
            os.makedirs(reg_folder, exist_ok=True) 
            

            for idx, url in enumerate(image_urls):
                url = url.strip() 
                if not url.startswith(("http://", "https://")): 
                    url = urljoin("https://", url) 
                
        
                parsed_url = urlparse(url)

       
                if not parsed_url.scheme or not parsed_url.netloc:
                    print(f"Invalid URL skipped: {url}") 
               
                else:
                
              
                    try:
                 
                        response = requests.get(url, stream=True) 
                        response.raise_for_status()
                        
                    
                        file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                       
                        file_extension = file_name.split(".")[-1]
                        
                 
                        if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                            file_name = f"image_{idx + 1}.jpg" 
                        
                        full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}.jpg")
                        
               
                        with open(full_file_name, 'wb') as f:
                            for chunk in response.iter_content(1024):
                                f.write(chunk)

            
                        add_watermark_to_image(full_file_name)
                        
                        print(f"Downloaded: {full_file_name}")
                    except Exception as e:
                        print(f"Failed to download {url} for {reg_no}: {e}")
# cond images
def download_cond(data, main_folder="Damaged_images"): 
    

    os.makedirs(main_folder, exist_ok=True)
    

    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        
        if pd.isna(row["Damaged_images"]) or not str(row["Damaged_images"]).strip():
            print(f"Skipping {reg_no} due to missing images.")
           
        else:
        
            image_urls = row["Damaged_images"].split(", ") 
            
     
            reg_folder = os.path.join(main_folder, reg_no) 
            os.makedirs(reg_folder, exist_ok=True) 
            
 
            for idx, url in enumerate(image_urls):
                url = url.strip() 
                if not url.startswith(("http://", "https://")):
                    url = urljoin("https://", url) 
                

                parsed_url = urlparse(url)

                if not parsed_url.scheme or not parsed_url.netloc:
                    print(f"Invalid URL skipped: {url}") 
 
                else:
                

                    try:
              
                        response = requests.get(url, stream=True) 
                        response.raise_for_status()
                        
                    
                        file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                    
                        file_extension = file_name.split(".")[-1]
                        
                    
                        if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                            file_name = f"image_{idx + 1}.jpg" 
                       
                        full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}.jpg")
                        
                    
                        with open(full_file_name, 'wb') as f:
                            for chunk in response.iter_content(1024):
                                f.write(chunk)

                        add_watermark_to_image(full_file_name)
                        
                        print(f"Downloaded: {full_file_name}")
                    except Exception as e:
                        print(f"Failed to download {url} for {reg_no}: {e}")

# reports
def download_reports(data, main_folder="Inspection Report"): 
    

    os.makedirs(main_folder, exist_ok=True)
    

    for index, row in data.iterrows():
        reg_no = row["Reg"] 
        report_urls = row["Inspection Report"]
        

        if not report_urls or pd.isna(report_urls):
            print(f"Missing Inspection Report of {reg_no}")

            
        else:
            if not report_urls.startswith(("http://", "https://")): 
                report_urls = urljoin("https://", report_urls) 
            

            parsed_url = urlparse(report_urls)
            
            try:
                response = requests.get(report_urls, stream=True) 
                response.raise_for_status() 
                file_name = os.path.basename(parsed_url.path) or f"inspec_repo.pdf"

   
                file_extension = file_name.split(".")[-1]
                
          
                if file_extension not in ["pdf", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"inspec_repo.pdf" 
                
         
                full_file_name = os.path.join(main_folder, f"{reg_no}.pdf")
                
             
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): 
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {report_urls} for {reg_no}:")

def start_funcs():
    thread1 = threading.Thread(target=download_images , args=(reg_img,))
    thread2 = threading.Thread(target=download_cond , args=(cond_img,))
    thread3 = threading.Thread(target=download_reports , args=(report,))

    thread1.start()
    thread2.start()
    thread3.start()
    
    thread1.join()
    thread2.join()
    thread3.join()

if __name__ == "__main__":
    start_funcs()


Skipping BJ67YMP due to missing images.
Skipping BJ67YMP due to missing images.
Downloaded: Inspection Report\BJ67YMP.pdf
Watermark added to Images\BT20ONV\BT20ONV_1.jpg
Downloaded: Images\BT20ONV\BT20ONV_1.jpg
Watermark added to Images\BT20ONV\BT20ONV_2.jpg
Downloaded: Images\BT20ONV\BT20ONV_2.jpg
Watermark added to Images\BT20ONV\BT20ONV_3.jpg
Downloaded: Images\BT20ONV\BT20ONV_3.jpg
Watermark added to Damaged_images\BT20ONV\BT20ONV_1.jpg
Downloaded: Damaged_images\BT20ONV\BT20ONV_1.jpg
Watermark added to Images\BT20ONV\BT20ONV_4.jpg
Downloaded: Images\BT20ONV\BT20ONV_4.jpg
Downloaded: Inspection Report\BT20ONV.pdf
Watermark added to Damaged_images\BT20ONV\BT20ONV_2.jpg
Downloaded: Damaged_images\BT20ONV\BT20ONV_2.jpg
Watermark added to Images\BT20ONV\BT20ONV_5.jpg
Downloaded: Images\BT20ONV\BT20ONV_5.jpg
Downloaded: Inspection Report\CN65PXM.pdf
Watermark added to Images\BT20ONV\BT20ONV_6.jpg
Downloaded: Images\BT20ONV\BT20ONV_6.jpg
Watermark added to Images\BT20ONV\BT20ONV_7.jpg
Do